In [89]:
# Preprocessing script (run once)
import sqlite3
import pandas as pd

# Load the raw dataset
df_lstat = pd.read_excel("datasets/Ladesaeulenregister_SEP.xlsx", header=10)
df_postcode = pd.read_csv("datasets/Postcode_BerlinDistricts.csv")

In [90]:
# Clean and filter the data
df_lstat['Ort'] = df_lstat['Ort'].str.strip().str.lower()
df_lstat = df_lstat.loc[df_lstat['Ort'] == 'berlin']



df_lstat = df_lstat.dropna(subset=['Breitengrad', 'Längengrad'])
df_lstat['Breitengrad'] = df_lstat['Breitengrad'].str.replace(',', '.').astype(float)
df_lstat['Längengrad'] = df_lstat['Längengrad'].str.replace(',', '.').astype(float)
df_lstat = df_lstat[(df_lstat['Breitengrad'].between(-90, 90)) & (df_lstat['Längengrad'].between(-180, 180))]
df_lstat['Address'] = df_lstat['Straße'] + ' ' + df_lstat['Hausnummer'].astype(str)
df_lstat['id'] = range(1, len(df_lstat) + 1)

In [91]:
df_lstat.columns

Index(['Betreiber', 'Anzeigename (Karte)', 'Straße', 'Hausnummer',
       'Adresszusatz', 'PLZ', 'Ort', 'Kreis/kreisfreie Stadt', 'Bundesland',
       'Breitengrad', 'Längengrad', 'Inbetriebnahmedatum',
       'Nennleistung Ladeeinrichtung [kW]', 'Art der Ladeeinrichung',
       'Anzahl Ladepunkte', 'Steckertypen1', 'P1 [kW]', 'Public Key1',
       'Steckertypen2', 'P2 [kW]', 'Public Key2', 'Steckertypen3', 'P3 [kW]',
       'Public Key3', 'Steckertypen4', 'P4 [kW]', 'Public Key4',
       'Steckertypen5', 'P5 [kW]', 'Public Key5', 'Steckertypen6', 'P6 [kW]',
       'Public Key6', 'Address', 'id'],
      dtype='object')

In [92]:
df_lstat = df_lstat.rename(columns={
    'Anzeigename (Karte)': 'Name',
    'Breitengrad': 'Latitude',
    'Längengrad': 'Longitude'
})
df_lstat['id'] = range(1, len(df_lstat) + 1)

df_lstat['Name'] = df_lstat['Name'].fillna("No Company Listed")
df_lstat['PLZ'] = df_lstat['PLZ'].astype(str).str.zfill(5)

In [93]:
df_postcode = df_postcode.rename(columns={
    'Postcode': 'PLZ'
})

df_postcode['PLZ'] = df_postcode['PLZ'].astype(str).str.zfill(5)


In [113]:
df_postcode.to_csv('datasets/Postcode_BerlinDistricts.csv', index=False)

In [105]:
df_postcode.head(1)

,District,PLZ
0,Mitte,10115


In [61]:
merged_df = df_lstat.merge(df_postcode, on='PLZ', how='inner')

# Replace 'Old_Value' with 'New_Value' in-place
merged_df['Ort'] = merged_df['District']

# Drop the 'New_Value' column
merged_df.drop('District', axis=1, inplace=True)

In [73]:
merged_df.head(1)

,Betreiber,Name,Straße,Hausnummer,Adresszusatz,PLZ,Ort,Kreis/kreisfreie Stadt,Bundesland,Latitude,...,P4 [kW],Public Key4,Steckertypen5,P5 [kW],Public Key5,Steckertypen6,P6 [kW],Public Key6,Address,id
0,Berliner Stadtwerke KommunalPartner GmbH,Berliner Stadtwerke KommunalPartner GmbH,Koppenplatz,11,NaN,10115,Mitte,Kreisfreie Stadt Berlin,Berlin,52.528513,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Koppenplatz 11,7


In [9]:



# Save the processed data to a SQLite database
conn = sqlite3.connect('charging_stations.db')
df_lstat[['id', 'Name', 'Address', 'Latitude', 'Longitude', 'PLZ']].to_sql('stations', conn, if_exists='replace', index=False)
conn.close()

In [11]:
json_string = df_lstat.to_json(orient='records') 

# Save the JSON string to a file
with open('data.json', 'w') as f:
    f.write(json_string)


In [77]:
with open('./output.geojson', 'r') as f:
    geojson_data = json.load(f)

for feature in geojson_data['features']:
    plz = feature['properties']['PLZ']
    if plz in plz_to_ort:
        feature['properties']['Ort'] = plz_to_ort[plz]

In [109]:
a = pd.read_csv("datasets/Postcode_BerlinDistricts.csv")
a.head(1)

,District,Postcode
0,Mitte,10115


In [121]:
import csv
import json

# Load the CSV file into a dictionary
csv_file_path = 'datasets/Postcode_BerlinDistricts.csv'
plz_district_dict = {}

with open(csv_file_path, mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        plz_district_dict[str(row['PLZ'])] = row['District']

print("CSV Data Loaded:", plz_district_dict)

# Load the GeoJSON file
geojson_file_path = 'output.geojson'
with open(geojson_file_path, 'r') as geojson_file:
    geojson_data = json.load(geojson_file)

# Update the "Ort" field in the GeoJSON file
for feature in geojson_data['features']:
    plz = str(feature['properties']['PLZ'])
    if plz in plz_district_dict:
        print(f"Updating PLZ {plz} to District {plz_district_dict[plz]}")
        feature['properties']['Ort'] = plz_district_dict[plz]
    else:
        print(f"PLZ {plz} not found in CSV file")

# Save the updated GeoJSON file
updated_geojson_file_path = 'updated_output.geojson'
with open(updated_geojson_file_path, 'w') as updated_geojson_file:
    json.dump(geojson_data, updated_geojson_file, indent=4)

print(f"Updated GeoJSON file saved to {updated_geojson_file_path}")

CSV Data Loaded: {'10115': 'Mitte', '10559': 'Mitte', '13355': 'Mitte', '10117': 'Mitte', '10623': 'Charlottenburg-Wilmersdorf', '13357': 'Mitte', '10119': 'Pankow', '10785': 'Tempelhof-Schoneberg', '13359': 'Mitte', '10178': 'Mitte', '10787': 'Tempelhof-Schoneberg', '13405': 'Reinickendorf', '10179': 'Friedrichshain-Kreuzberg', '10963': 'Friedrichshain-Kreuzberg', '13407': 'Reinickendorf', '10435': 'Pankow', '10969': 'Friedrichshain-Kreuzberg', '13409': 'Reinickendorf', '10551': 'Mitte', '13347': 'Mitte', '10553': 'Charlottenburg-Wilmersdorf', '13349': 'Mitte', '10555': 'Mitte', '13351': 'Mitte', '10557': 'Mitte', '13353': 'Charlottenburg-Wilmersdorf', '10967': 'Neukolln', '10243': 'Friedrichshain-Kreuzberg', '10245': 'Friedrichshain-Kreuzberg', '10997': 'Friedrichshain-Kreuzberg', '10247': 'Pankow', '10999': 'Friedrichshain-Kreuzberg', '10249': 'Pankow', '12045': 'Neukolln', '10367': 'Lichtenberg', '10961': 'Friedrichshain-Kreuzberg', '10965': 'Neukolln', '13053': 'Lichtenberg', '131